In [1127]:
%matplotlib widget

from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

from fimpylab import LightsheetExperiment

from matplotlib import  pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, fictive_trajectory_and_fit, fit_phase_neurons,qap_sorting_and_phase
from circle_fit import hyper_fit

In [1252]:
plt.close("all")
path = Path("/Users/luigipetrucco/Desktop/batch_210922/210927_f3eyes_gainmod")
traces = fl.load(path / "filtered_traces.h5", "/detr")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
traces[np.isnan(traces)] = 0

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LightsheetExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

t_start = 150
time_slices_dict = dict(clol=(t_start*fn, 8000), 
                        cwccw=(t_start*fn, traces.shape[0] // 2),
                        gainmod=(t_start*fn, traces.shape[0] - t_start*fn))
t_lims = time_slices_dict[path.name.split("_")[-1]]
t_slice = slice(*t_lims)

In [1253]:
sel = (df["t_start"] > 10) & (df["t_start"] < 2020)
plt.figure(figsize=(5, 2))
plt.hist(df["bias"], 10, density=True, zorder=-100)
plt.scatter(df["bias"], df["peak_vig"], 100, c=cols[1])
plt.ylim(0., 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.


(0.0, 3.0)

In [1254]:
plt.figure(figsize=(3, 3))
plt.imshow(anat.mean(0), vmax=10, vmin=0)
#plt.scatter(coords[:, 1], coords[:, 2], c=(0.9,)*3)
s1 = 100
s2 = 280
plt.axvline(s1)
plt.axvline(s2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1255]:
sel_to_nan = (coords[:, 2] < s1) | (coords[:, 2] > s2)
traces[:, sel_to_nan] = 0
cc_motor[sel_to_nan] = np.nan
cc_motor_integr[sel_to_nan] = np.nan

In [1256]:
cc_wnd = 500
i_array = np.arange(t_slice.start, t_slice.stop, cc_wnd*fn)
cc_mats = np.zeros((traces.shape[1], traces.shape[1], len(i_array)))

for n, i in enumerate(i_array):
    cc_mats[:, :, n] = np.corrcoef(traces[i:i + cc_wnd*fn, :].T)
corr_mat = np.nanmean(cc_mats, 2)

selection_arr = np.zeros(traces.shape[1])

f = plt.figure(figsize=(3, 3))
x = np.arange(-0.2, np.nanmax(cc_motor), 0.05)
s = plt.scatter(cc_motor, cc_motor_integr, s=10, c=selection_arr, vmin=0, vmax=1)

l_plot = plt.plot(x, x*0.2 + 0.15)
l_max = plt.axvline(1)
l_min = plt.axhline(0)

@widgets.interact(c=(0.05, 2, 0.05), o=(-0.5, 1, 0.02), mot_max=(0, 1, 0.05),
                 integr_min=(0, 1, 0.02), max_corr=(-1, 0, 0.05))
def update(o=0.3, c=0.2, mot_max=1, integr_min=0, max_corr=-0.7):
    l_plot[0].set_data(x, x*c + o)
    print(cc_motor_integr.shape, cc_motor.shape, (cc_motor*c + o).shape)
    selection_arr[:] = (cc_motor_integr > cc_motor*c + o) & \
                       (np.abs(cc_motor) < mot_max) & \
                       (np.abs(cc_motor_integr)> integr_min) | \
                       ((np.nanmin(corr_mat, 0) < max_corr) & \
                        (np.abs(cc_motor) < mot_max) & \
                        (np.abs(cc_motor_integr) > integr_min)) 
    l_max.set_xdata(mot_max)
    l_min.set_ydata(integr_min)

    
    s.set_array(selection_arr)

plt.ylim(-0.15, 0.4)
plt.xlim(-0.3, 1.01)
plt.xlabel("cc. traces - motor regressor")
plt.ylabel("cc. d(traces)/dt - regressor")
sns.despine()

/Users/luigipetrucco/anaconda3/envs/rplab/lib/python3.8/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/luigipetrucco/anaconda3/envs/rplab/lib/python3.8/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
<ipython-input-1256-c4b39c380f61>:7: RuntimeWarning: Mean of empty slice
  corr_mat = np.nanmean(cc_mats, 2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.3, description='o', max=1.0, min=-0.5, step=0.02), FloatSlider(value…

In [1257]:
selected = np.argwhere(selection_arr)[:, 0]

In [1308]:
# selected = fl.load(path / "selected.h5")
pcaed_t, phase_t = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, selected].T)
hf_c = hyper_fit(pcaed_t)
pcaed_t_all, _ = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, :].T)


plt.figure(figsize=(7, 3))
thr = 35
sel = (pcaed_t[:, 0]**2+pcaed_t[:, 1]**2)**(1/2) > thr
plt.scatter(pcaed_t[:, 0], pcaed_t[:, 1], c=sel)
plt.scatter(pcaed_t_all[:, 0], pcaed_t_all[:, 1], edgecolor="k", facecolor="none", lw=0.2)
plt.axis("equal")

#selected = selected[sel]
# pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])
#pcaed_spont, phase_spont = pca_and_phase(traces[t_slice, selected], traces[t_slice, selected])
pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])

x1 = hf_c[2]*np.cos(np.linspace(0, 2*np.pi, 100)) + hf_c[0]
x2 = hf_c[2]*np.sin(np.linspace(0, 2*np.pi, 100)) + hf_c[1]

plt.plot(x1, x2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1272]:
new_selection_arr = (np.abs(np.sqrt((pcaed_t_all[:, 0] - hf_c[0])**2 + (pcaed_t_all[:, 1] - hf_c[1])**2) - hf_c[2]) < 15) | \
    ((np.sqrt((pcaed_t_all[:, 0] - hf_c[0])**2 + (pcaed_t_all[:, 1] - hf_c[1])**2) - hf_c[2]) > 0)
selected = np.argwhere(new_selection_arr)[:, 0]

In [1274]:
plt.figure(figsize=(7, 2.5))
plt.plot(traces[:, selected] + 4)
print(len(selected))
plt.plot(beh_df["t"]*fn, beh_df["tail_sum"])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

141


In [1303]:
# plt.close("all")
pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])
mot_t_slice = slice(traces.shape[0] // 2, traces.shape[0])
f, axs = plt.subplots(1, 3, figsize=(7., 3.), sharex=True, sharey=True)
# plt.subplot(1,2,1)
phase = np.angle((pcaed[:, 0] - 2) + 1j * (pcaed[:, 1] - 3))
for i, s in enumerate([t_slice, mot_t_slice,  slice(None, None, None)]):
    
    axs[i].plot(pcaed[s, 0], pcaed[s, 1], 
             c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
    axs[i].scatter(pcaed[s, 0], pcaed[s, 1], 
                     c=phase[s], lw=0.5, s=5, cmap="twilight",) 
    # plt.axis("equal")
sns.despine()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1300]:
%%time
import os
perm, com_phase = qap_sorting_and_phase(traces[:, selected], t_lims=t_lims)

phases_neuron, _ = fit_phase_neurons(traces[t_slice, selected], phase[t_slice])
perm_pca = np.argsort(phases_neuron)
os.system('say "Fit completed"')

100%|██████████| 94/94 [00:00<00:00, 180.02it/s]


CPU times: user 33 s, sys: 760 ms, total: 33.8 s
Wall time: 10.3 s


0

In [1307]:
l = 2
f, axs = plt.subplots(2,2, figsize=(7, 7), sharey=True)
# plt.subplot(121)
axs[0, 0].imshow(np.corrcoef(traces[t_slice, selected].T)[perm, :][:, perm], 
           vmax=1, vmin=-1, cmap="RdBu_r", aspect="auto")

axs[0, 1].imshow(traces[:, selected[perm]].T, cmap="gray_r", interpolation="none",
              aspect="auto", vmin=-l, vmax=l)

axs[1, 0].imshow(np.corrcoef(traces[t_slice, selected].T)[perm_pca, :][:, perm_pca], 
           vmax=1, vmin=-1, cmap="RdBu_r", aspect="auto")

axs[1,1].imshow(traces[:, selected[perm_pca]].T, cmap="gray_r", interpolation="none",
              aspect="auto", vmin=-l, vmax=l)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1299]:
# old_selected = selected.copy()
rm_from_selected = np.array([11, 13, 18, 19, 21, 23, 25, 28, 30, 32, 37])
selected[perm[rm_from_selected]] = -1
##for i in s:# [23,  64,  82, 110, 152, 193,  87, 127, 143,  57,  33, 226, 201,  89, 155,  92,  34]:
#    if i is not " " and i is not "":
#        selected[perm == int(i)] = -1
selected = selected[selected > 0]

In [1302]:
unwrapped_phase = np.unwrap(phase)
unwrapped_com_phase = np.unwrap(com_phase)

traj, params = fictive_trajectory_and_fit(unwrapped_phase, df, min_bias=0.1)
print(params)

plt.figure(figsize=(7, 3))
plt.scatter(np.arange(len(traj[:])), zscore(unwrapped_phase), 
            c=phase[:], cmap="twilight", s=2)
plt.scatter(np.arange(len(traj[:])), zscore(unwrapped_com_phase), 
            c=com_phase[:], cmap="twilight", s=0.2)
plt.plot(-zscore(traj), c=cols[1])

[-7.34254097 -2.567252  ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1309]:
f, axs = plt.subplots(1, 2, figsize=(6, 3))
s = coords[:, 0] > 0
selection = np.full(coords.shape[0], False)
selection[selected] = True
all_phases = np.zeros(coords.shape[0])
all_phases[selected] = phases_neuron

all_perm = -np.ones(coords.shape[0])
all_perm[selected] = perm

axs[0].scatter(coords[s, 1], coords[s, 2], c=(0.5,)*3)
axs[0].scatter(coords[s, :][selection[s], 1], coords[s, :][selection[s], 2],
            c=all_phases[s][selection[s]], cmap="twilight")
axs[0].axis("equal")
axs[0].axis("off")

axs[1].scatter(coords[s, 1], coords[s, 2], c=(0.5,)*3)
axs[1].scatter(coords[s, :][selection[s], 1], coords[s, :][selection[s], 2],
            c=np.linspace(-np.pi, np.pi, sum(all_perm[s] >= 0)+1)[np.argsort(all_perm[s][all_perm[s] >= 0])] , 
               cmap="twilight")
axs[1].axis("equal")
axs[1].axis("off")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.


(11.974509803921569, 254.53529411764706, 39.21630097087379, 302.0887475728155)

In [ ]:
plt.figure(figsize=(8, 3.5))
plt.subplot(221)
#sel = # (coords[selected, 1] < 50 ) #& (coords[selected, 1] < 209) & \
        # (coords[selected, 2] > 206 ) & (coords[selected, 2] < 250)
sel = (coords[selected, 2] < 167 )
plt.scatter(coords[selected, 1], 
            np.linspace(-np.pi, np.pi, perm.max()+1)[np.argsort(perm)] )
plt.subplot(222)
plt.scatter(coords[selected, 1], perm)

plt.subplot(223)
plt.scatter(coords[selected[sel], 1], 
            np.linspace(-np.pi, np.pi, perm.max()+1)[np.argsort(perm)][sel] )
plt.subplot(224)
plt.scatter(coords[selected[sel], 1], perm[sel])
print(perm[sel])

In [1310]:
fl.save(path / "selected.h5", selected)

In [77]:
from sklearn.decomposition import PCA
from circle_fit import hyper_fit
comp0, comp1 = 0, 1

traces_fit = traces[2000:8000, selected].T
traces_transform = traces_fit
if traces_transform is None:
    traces_transform = traces_fit

# Compute PCA and transform traces:
pca = PCA(n_components=5).fit(traces_fit)
pcaed_t = pca.transform(traces_transform)

# Fit circle:
hf_c = hyper_fit(pcaed[:, [comp0, comp1]])

# Compute phase, after subtracting center of the circle
phase_t = np.angle((pcaed_t[:, 0] - hf_c[0]) + 1j * (pcaed_t[:, 1] - hf_c[1]))

plt.figure(figsize=(7, 3))
plt.scatter(pcaed_t[:, 0], pcaed_t[:, 1], c=phase_t, cmap="twilight")
plt.axis("equal")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-89.43453865051269, 93.88069801330566, -60.719933319091794, 75.67397384643554)

In [78]:
# plt.close("all")
plot_t_s = 10
plot_t_pts = int(plot_t_s * fn)

rot_wnd_s = 1
rot_wnd_pts = int(rot_wnd_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
# [ax.set_xlabel("PC 1") for ax in axs]
# axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list[:]:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            crop_seg = crop_seg - crop_seg[0, :]
            th = np.arctan2(crop_seg[rot_wnd_pts, 0] - crop_seg[0, 0], 
                            crop_seg[rot_wnd_pts, 1] - crop_seg[0, 1])

            rot_mat = np.array([[np.cos(th), np.sin(th)], [-np.sin(th), np.cos(th)]]).T 
            crop_seg = (rot_mat @ crop_seg[:, :].T).T
            ax.plot(crop_seg[::3, 0], crop_seg[::3, 1], c=(0.4,)*3, lw=0.3, zorder=-100)
            ax.scatter(crop_seg[::3, 0], crop_seg[::3, 1], c=np.arange(len(crop_seg[::3, 1])), cmap=cmap, s=1)
            #cp = color_plot(crop_seg[::3, 0], crop_seg[::3, 1], ax=ax,
             #          cmap=cmap, lw=1)
        except IndexError:
            pass
    cbars.append(add_cbar((0.93, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
    ax.axvline(0, lw=0.5, c=(0.4,)*3)
    ax.axhline(0, lw=0.5, c=(0.4,)*3)

plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.95])
cbars[0].set_ticklabels([0, plot_t_s])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'idx_l' is not defined

In [730]:
plt.close("all")
plot_t_s = 8
plot_t_pts = int(plot_t_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
[ax.set_xlabel("PC 1") for ax in axs]
axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            # crop_seg = crop_seg - crop_seg[0, :]
            #cp = color_plot(crop_seg[::3, 0], crop_seg[::3, 1], ax=ax,
            #           cmap=cmap, lw=1)
            ax.plot(crop_seg[::3, 0], crop_seg[::3, 1], c=(0.4,)*3, lw=0.3, zorder=-100)
            ax.scatter(crop_seg[::3, 0], crop_seg[::3, 1], c=np.arange(len(crop_seg[::3, 1])), cmap=cmap, s=1)
            
        except (IndexError, ValueError):
            pass
    cbars.append(add_cbar((0.92, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
        
plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.8])
cbars[0].set_ticklabels([0, plot_t_s])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-730-ef9dc3fac9ea>:32: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-730-ef9dc3fac9ea>:32: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()


In [501]:
path

PosixPath('/Volumes/Shared/experiments/E0040_motions_cardinal/v15_playback/210511_f3b_natmov')

In [121]:
fl.save(path / "selected_new.h5", selected)

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0040_motions_cardinal_old/v15_playback")
all_list = list(master_path.glob("[0-9]*_f[0-9]*"))
all_valid = list(master_path.glob("[0-9]*_f[0-9]*/selected.h5"))
print(f"{len(all_valid)}/{len(all_list)}")

In [1160]:
plt.close("all")
plot_t_s = 8
plot_t_pts = int(plot_t_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
[ax.set_xlabel("PC 1") for ax in axs]
axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            # crop_seg = crop_seg - crop_seg[0, :]
            cp = color_plot(crop_seg[:, 0], crop_seg[:, 1], ax=ax,
                       cmap=cmap, lw=1)
        except IndexError:
            pass
    cbars.append(add_cbar((0.92, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
        
plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.8])
cbars[0].set_ticklabels([0, plot_t_s])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-1160-f1976b198914>:29: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-1160-f1976b198914>:29: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  plt.tight_layout()
